<a href="https://colab.research.google.com/github/ednavivianasegura/H_HSGP4_Ariadna/blob/main/Ariadna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# @title **Clonar carpeta GitHub** (Para acceder a módulos y datasets) { form-width: "5%", display-mode: "form" }
# Clona el repositorio
try:
    !git clone https://github.com/ednavivianasegura/H_HSGP4_Ariadna.git
except:
    pass

# Cambiar al directorio "Curso_PLN"
os.chdir("H_HSGP4_Ariadna")

Cloning into 'H_HSGP4_Ariadna'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 34 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (34/34), 80.27 MiB | 6.67 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Updating files: 100% (27/27), done.


In [7]:
!git clone https://gitlab.com/libeigen/eigen.git

Cloning into 'eigen'...
remote: Enumerating objects: 124182, done.
remote: Counting objects: 100% (336/336), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 124182 (delta 191), reused 309 (delta 181), pack-reused 123846
Receiving objects: 100% (124182/124182), 105.06 MiB | 20.41 MiB/s, done.
Resolving deltas: 100% (102808/102808), done.


In [3]:
!pip install sinfo

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 1.4 MB/s eta 0:00:00
  Created wheel for sinfo: filename=sinfo-0.3.4-py3-none-any.whl size=7880 sha256=3c5a1330c27f49cde8f786142b52cc142201e16f9159c69cbf8ce0ad54270e36
  Stored in directory: /root/.cache/pip/wheels/ae/fe/9d/eb4b47396d5c94b8ad82a5aa9f905c56c981deb4e532329f72
Successfully built sinfo


In [31]:
import os, time, sys, gc, random, json, re, shutil
from distutils.util import strtobool
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
import subprocess
import platform
#Validar requisitos del script
from sinfo import sinfo
import FuncionesAriadnaDL_Noaleatorio as FAVCI

Plataforma=platform.platform()
print(Plataforma)
print(sinfo())

Linux-6.1.58+-x86_64-with-glibc2.35
The `sinfo` package has changed name and is now called `session_info` to become more discoverable and self-explanatory. The `sinfo` PyPI package will be kept around to avoid breaking old installs and you can downgrade to 0.3.2 if you want to use it without seeing this message. For the latest features and bug fixes, please install `session_info` instead. The usage and defaults also changed slightly, so please review the latest README at https://gitlab.com/joelostblom/session_info.
-----
FuncionesAriadnaDL_Noaleatorio      NA
numpy                               1.25.2
pandas                              1.5.3
sinfo                               0.3.4
tensorflow                          2.15.0
-----
IPython             7.34.0
jupyter_client      6.1.12
jupyter_core        5.7.1
notebook            6.5.5
-----
Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Linux-6.1.58+-x86_64-with-glibc2.35
2 logical CPU cores, x86_64
-----
Session informatio

In [55]:
# @title Entradas:
# Modeling=True if Tensorflow is to be used (The models are created), Modeling=False otherwise
Modeling = True # @param {type:"boolean"}

fore_set = ["train","test"] # @param ["[\"train\"]", "[\"valid\"]", "[\"test\"]", "[\"train-valid\"]", "[\"valid-test\"]", "[\"train-valid-test\"]", "[\"train\",\"test\"]", "[\"train\",\"valid\"]", " [\"train\",\"valid\",\"test\"]", "[\"train\",\"test\",\"train-valid-test\"]", "[\"train\",\"test\",\"valid-test\",\"train-valid-test\"]", "[\"train-test\"]", "[\"train\",\"train-test\"]", "[\"train\",\"valid-test\",\"test\"]"] {type:"raw"}

# In BasesAModelar you can set up an arrangement of bases to perform the learning process.
# Intervalo = True is to define an interval, Intervalo = False is to indicate that it is an array of specific series.

Intervalo = False # @param {type:"boolean"}
BasesAModelar = [1,2]

# Type of variables:
var_set = "hill" # @param ["hill", "dln", "orb", "equi", "cart"]

# Especific name for the experiment (to be included in folder name)
experimento = "Prueba" # @param {type:"string"}

fachl = 0.5 # @param {type:"slider", min:0, max:10, step:0.1}

#TYPE OF HYPER-PARAMETER SEARCH
method = "Cartesian" # @param ["Cartesian", "Random", "Especific model"]

# Effective (without starting vector)
train_set = 0 # @param {type:"number"}

valid_set = 0 # @param {type:"number"}

test_set = 0 # @param {type:"number"}

# Resolution: [min]
resol = 10  # @param {type:"number"}

# Cross-val (if nfolds!:0 -> valid_set:0)
nfolds = 0   # @param {type:"number"}


# Remove trend
rem_trend = False # @param {type:"boolean"}

path_data = "data/" # @param {type:"string"}


var_model = "theta" # @param {type:"string"}

var_order= ["t","r","theta","v","R","THETA","N"]

#############################################################################################################
###### SAMPLING CRITERIA
#############################################################################################################
#Sampling proportional to the stratum size "SampleMethod":1 is chosen, "SampleMethod":0 otherwise.
SampleMethod = True  # @param {type:"boolean"}
# if SampleMethod=1, one of the hyperparameters should be chosen for stratification, the variable name, must be equal to the name of the hyperparameter defined in ParametersDL
VariableToStratify = "opt" # @param {type:"string"}

##########################################################################################################
###### HYPER-PARAMETERS DEEP LEARNING
##########################################################################################################
Parametros = {
#ACTIVATION FUNCTION
  "FuncAct":["tanh","linear","elu","relu"],
#BATCH SIZE
  "BachSize":[64,128,256,512],
#NUMBER OF NEURONS IN THE INPUT LAYER
  "NumNeurons":[8,16,32,64,128,256],
#OPTIMIZERS
  "opt":["Adam", "Rmsprop","Adadelta", "Nadam"],
#loss:
  "loss":["mape"],
#Learning rate:
  "LR":[1e-05],
#Monitor
  "monitor" : ["val_loss"],
#NUMBER OF HIDDEN LAYERS
  "HiddenLayers":[2]
    }

#Specific configuration:
CONFIG =[
{"BachSize": 64, "NumNeurons": 16, "opt": "Rmsprop", "loss": "mape", "LR": 0.0001, "monitor": "val_loss", "FuncAct0": "tanh", "FuncAct1": "elu", "HiddenLayers": 2},
{"BachSize": 256, "NumNeurons": 64, "opt": "Nadam", "loss": "mape", "LR": 0.0001, "monitor": "val_loss", "FuncAct0": "linear", "FuncAct1": "tanh", "HiddenLayers": 2},
{"BachSize": 256, "NumNeurons": 32, "opt": "Nadam", "loss": "mape", "LR": 0.0001, "monitor": "val_loss", "FuncAct0": "linear", "FuncAct1": "tanh", "HiddenLayers": 2}
]


# Keplerian period: [h] (0 for AUTO)
kep_period      = 0

# Possibilities: "num", "span" [h], "rev" [rev]
input_unit = "rev"

inputI = 2

output_num=1

sets_unit=input_unit

# Step betconsecutive vectors (1 to maximize no. vectors)
vect_step      = 1

#max number of epochs
epocas = 500  # @param {type:"number"}
#Patience to find number of Epochs
stop_tolerance = 60  # @param {type:"number"}
#Number of models you create with the same architecture to pick the minimum,  must be greater than 0
n_repeats = 10   # @param {type:"number"}


In [56]:
I_CreaInfo=time.time()
if Intervalo==True:
    BasesAModelar=range(BasesAModelar[0],BasesAModelar[1]+1)
else:
    BasesAModelar=BasesAModelar

for h1 in BasesAModelar:

    print(var_set+"-"+str(h1)+" Time series modelling.")
    ##########################################################################################################
    ###### NAME OF EPHEMERID TO BE STUDIED
    ##########################################################################################################
    id_expM = var_set+str(h1)
    ##########################################################################################################
    ###### CREATE NAME OF DIRECTORY WITH PARAMETERS
    ##########################################################################################################
    experiment=str(experimento)+"_"+"Fachl"+str(fachl) +"Me"+str(method)+"TrS"+str(train_set) + "VaS"+str(valid_set)+"TeS"+str(test_set)+"Res"+str(resol)+"Nf"+str(nfolds)+"Rt"+str(rem_trend) +"VarS"+str(var_set)+"Ep"+str(epocas)+"St"+str(stop_tolerance)+"VarM"+str(var_model)
    ###########################################################################################################
    ###### Create folder name
    ###########################################################################################################
    Conid_exp = id_expM + experiment
    cwd=os.getcwd()
    cwd2=cwd
    ##########################################################################################################
    ###### Configuration of files and folders
    ##########################################################################################################
    try:
      os.stat('KerasPruebasFinal/')
    except:
      os.mkdir('KerasPruebasFinal/')

    path= 'KerasPruebasFinal/'+Conid_exp+'/'

    path_results           = "results/"
    path_out               = "output/"
    ResumenEjecucion       = path+"output/Log.txt"
    path_plot              = "figs/"
    path_models            = "Modelos/"
    path_cpp               = "cpp/"
    file_obs               = 'obs'+id_expM+'.out'
    file_appr              = 'approx'+id_expM+'.out'
    file_BasicPlot         = id_expM +"_BasicPlot"
    file_plot_Decomp       = id_expM +"_Decomp"
    file_plot_Complete     = id_expM +"_Completo"
    fileCandidato          = path+path_results+'Candidatos.csv'
    path_path_models       = path+path_models
    path_file_obs          = path_data+"OBS/"+file_obs
    path_file_appr         = path_data+"APPROX/"+file_appr
    path_file_BasicPlot    = path+path_plot + file_BasicPlot +'.pdf'
    fileplotDetrending     = path+path_plot + file_plot_Decomp+".pdf"
    fileplotComplete       = path+path_plot + file_plot_Complete+".pdf"
    directorio             = ['figs','Modelos','output','results', "cpp"]

    #Grid Comfigurations:
    hp_all=FAVCI.DeepLerningModelConfigs(Parametros)
    NModTotal=len(hp_all)
    random.seed(2022)
    hp_all=random.sample(hp_all, len(hp_all))

    if method=="Cartesian":
        NModels=NModTotal
        hp_all=hp_all
        Metodo="Cartesiana"
    elif method=="Random":
        Metodo="Aleatoria"
        print("Hiper-parameter to sampling: "+ VariableToStratify)
        ListaDeOpciones=Parametros[VariableToStratify]
        hp_all=[y for x in ListaDeOpciones for y in hp_all if x==y[VariableToStratify]]
        StrVar=[str(hp_all).count("'"+VariableToStratify+"': '"+str(n)+"'") if type(n)==str else str(hp_all).count("'"+VariableToStratify+"': "+str(n)) for n in ListaDeOpciones]

        if SampleMethod==0:
            if NModels< len(StrVar):
                NModels=len(StrVar)
            else:
                NModels=FAVCI.roundBy(NModels, len(StrVar))
            indices=[]
            nPerHipePar=round(NModels/len(StrVar))
            a=0
            for i in StrVar:
                random.seed(2022)
                indices=indices+random.sample(range(a, i+a), nPerHipePar)
                a=i+a
            hp_all=[hp_all[index] for index in indices]
        else:
            indices=[]
            nPerHipePar=[round(NModels*(x/sum(StrVar))) for x in StrVar]
            a=0
            zipped = list(zip(StrVar,nPerHipePar))
            for i in zipped:
                print(i)
                random.seed(2022)
                X=random.sample(range(a, i[0]+a), i[1])
                indices=indices+X
                a=i[0]+a
            # indices.sort()
            hp_all=[hp_all[index] for index in indices]


    elif method==3:
        Metodo="Modelo Específico"
        # hp_all = [CONFIG]
        hp_all = CONFIG
        NModels=len(hp_all)

    print("Total number of possible models: "+ str(NModTotal)+"\n"+
          "Number of models to find: "+ str(len(hp_all)))

    ###########################################################################################################
    ###### FOLDER CREATION AND VALIDATION
    ###########################################################################################################
    # Check if a folder with the same name as the current experiment already exists.
    # If not, a folder with that name is created.
    ###########################################################################################################

    Verif=path_results

    if os.path.exists(path):

    ###############################################################
    # It is verified that the existing folder is of a complete or
    # finished experiment, otherwise it is deleted.
    # experiment, otherwise the folder is deleted.
    ###############################################################

        if os.path.exists(path+Verif) and len(path_path_models)==NModTotal:

            sub=1
            repeticion=""
            NuevoArchivo="False"


            while(NuevoArchivo=="False"):

    #############################################################################
    # If the existing folder has a completed experiment
    # then folders are searched for folders with the same experiment but
    # performed later. What is identified by (i), if found, is
    # verify that they are complete. Otherwise, they are deleted and the current
    # is assigned that name. "os.path.exists(path)" is checked
    # again since in each iteration of while, it changes due to "sub".
    #############################################################################

                if  os.path.exists(path):

                    if len(os.listdir(path+Verif))>0:
                        repeticion="(" +str(sub)+")" +'/'
                        path= 'KerasPruebasFinal/'+Conid_exp+repeticion
                        sub+=1

                    else:

                        print(f"Borrando directorio vacio 1 {path}."+'\n')
                        shutil.rmtree(path)
                        os.mkdir(path)
                        os.chdir(path)


                        for dir in directorio:
                            try:
                                os.stat(dir)
                            except:
                                os.mkdir(dir)

                        NuevoArchivo="True"
                        os.chdir(cwd2)

                else:

                    os.mkdir(path)
                    os.chdir(path)


                    for dir in directorio:
                            try:
                                os.stat(dir)
                            except:
                                os.mkdir(dir)
                    NuevoArchivo="True"
                    os.chdir(cwd2)



        elif os.path.exists(path+Verif) and len(path_path_models)<NModTotal:
            pass
        else:

            print(f"Borrando directorio vacio 2 {path}."+'\n')
            shutil.rmtree(path)
            os.mkdir(path)
            os.chdir(path)


            for dir in directorio:

                try:
                    os.stat(dir)
                except:
                    os.mkdir(dir)

            os.chdir(cwd2)
    else:

        os.mkdir(path)
        os.chdir('KerasPruebasFinal/'+Conid_exp)

        for dir in directorio:

            try:
                os.stat(dir)
            except:
                os.mkdir(dir)

        os.chdir(cwd2)


    if not os.path.isfile(path+path_out+"hp_all.csv"):
        pd.DataFrame(hp_all).to_csv(path+path_out+"hp_all.csv")
    #########################################################################
    #                End of Folder Verification                             #
    #########################################################################
    if not os.path.isfile(ResumenEjecucion):
        Start=str(
          "#############################################################################\n"+
          "Start modeling: \n"+
          "#############################################################################\n"+
          "Start time: "+ str(datetime.now()) +"\n"+
          "Folder Name: "+ str(Conid_exp) +"\n"+
          "Grid Search Method: " + str(Metodo) + "\n"+
          "Number of models: "+ str(NModels) +"\n"+
          "Maximum number of epochs: "+str(epocas)+ "\n"+
          "Stop tolerance: "+ str(stop_tolerance) +"\n"+
          "#############################################################################\n"+
          "#############################################################################\n"
          )
        with open(ResumenEjecucion, 'a') as f:
            print(Start
          , file=f)
        print(Start)

    ##########################################################################################################
    ###### CREATING DATABASES
    ##########################################################################################################
    I_bases=time.time()
    Bases=FAVCI.creaBases(var_order,path_file_obs,path_file_appr,resol,kep_period,var_set,input_unit,inputI,
                          output_num,nfolds,sets_unit,train_set,test_set,var_model,rem_trend,vect_step,
                          ResumenEjecucion,valid_set)

hill-1 Time series modelling.
DeepLerningModelConfigs function time:  0.0

Total number of possible models: 1536
Number of models to find: 1536


/content/H_HSGP4_Ariadna/Coordinates.py:421: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  eSinE = (np.multiply(pos,vel)).sum(axis=1) / np.sqrt(mu*a)        # e*sin(E)


NameError: name 'exit' is not defined